<a href="https://colab.research.google.com/github/TiagoBarantini/Pro_Eng_dados/blob/main/prepara%C3%A7%C3%A3o%2C_orquestra%C3%A7%C3%A3o_e_fluxos_de_dados_aula03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%bash
#instalação JAVA
apt-get update && apt-get install openjdk-8-jdk-headless -qq > /dev/null

#instalação do pyspark
pip install -q pyspark

#download dos dados utilizados
mkdir titanic
curl https://raw.githubusercontent.com/neylsoncrepalde/titanic_data_with_semicolon/main/titanic.csv -o titanic/titanic.csv



In [3]:
#definir uma variavel de ambiente para identificador local do JAVA
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

In [4]:
#Definir a sessão spark
from pyspark.sql import SparkSession
from pyspark.sql import functions as f

In [5]:
spark = SparkSession.builder.getOrCreate()

In [ ]:
#leitura dos dados
titanic = (
    spark
    .read
    .csv("titanic", header=True, sep=";", inferSchema=True)
  )

titanic.printSchema()

In [7]:
#registrat o dataset como uma view
titanic.createOrReplaceTempView('titanic')

In [9]:
spark.sql("""
  SELECT
    Name,Sex,Pclass,Age
  FROM titanic

""").show()

+--------------------+------+------+----+
|                Name|   Sex|Pclass| Age|
+--------------------+------+------+----+
|Braund, Mr. Owen ...|  male|     3|22.0|
|Cumings, Mrs. Joh...|female|     1|38.0|
|Heikkinen, Miss. ...|female|     3|26.0|
|Futrelle, Mrs. Ja...|female|     1|35.0|
|Allen, Mr. Willia...|  male|     3|35.0|
|    Moran, Mr. James|  male|     3|null|
|McCarthy, Mr. Tim...|  male|     1|54.0|
|Palsson, Master. ...|  male|     3| 2.0|
|Johnson, Mrs. Osc...|female|     3|27.0|
|Nasser, Mrs. Nich...|female|     2|14.0|
|Sandstrom, Miss. ...|female|     3| 4.0|
|Bonnell, Miss. El...|female|     1|58.0|
|Saundercock, Mr. ...|  male|     3|20.0|
|Andersson, Mr. An...|  male|     3|39.0|
|Vestrom, Miss. Hu...|female|     3|14.0|
|Hewlett, Mrs. (Ma...|female|     2|55.0|
|Rice, Master. Eugene|  male|     3| 2.0|
|Williams, Mr. Cha...|  male|     2|null|
|Vander Planke, Mr...|female|     3|31.0|
|Masselmani, Mrs. ...|female|     3|null|
+--------------------+------+-----

In [11]:
spark.sql("""
  SELECT
    Name,Sex,Pclass,Age
  FROM titanic
  WHERE
    Sex = 'male'
    AND Pclass = 1
""").show()

+--------------------+----+------+----+
|                Name| Sex|Pclass| Age|
+--------------------+----+------+----+
|McCarthy, Mr. Tim...|male|     1|54.0|
|Sloper, Mr. Willi...|male|     1|28.0|
|Fortune, Mr. Char...|male|     1|19.0|
|Uruchurtu, Don. M...|male|     1|40.0|
|Meyer, Mr. Edgar ...|male|     1|28.0|
|Holverson, Mr. Al...|male|     1|42.0|
|Ostby, Mr. Engelh...|male|     1|65.0|
|   Woolner, Mr. Hugh|male|     1|null|
|Harris, Mr. Henry...|male|     1|45.0|
|Stewart, Mr. Albe...|male|     1|null|
|Carrau, Mr. Franc...|male|     1|28.0|
|Chaffee, Mr. Herb...|male|     1|46.0|
|Goldschmidt, Mr. ...|male|     1|71.0|
|Greenfield, Mr. W...|male|     1|23.0|
|White, Mr. Richar...|male|     1|21.0|
|Porter, Mr. Walte...|male|     1|47.0|
|Baxter, Mr. Quigg...|male|     1|24.0|
|White, Mr. Perciv...|male|     1|54.0|
|Futrelle, Mr. Jac...|male|     1|37.0|
|  Giglio, Mr. Victor|male|     1|24.0|
+--------------------+----+------+----+
only showing top 20 rows



In [13]:
#agregaçoes

spark.sql("""
  SELECT
    AVG(Age) as med_idade,
    MIN(Age) as idade_minima,
    MAX(Age) as idade_maxima,
    STDDEV(Age) as des_ped_idade
  FROM titanic
""").show()

+-----------------+------------+------------+------------------+
|        med_idade|idade_minima|idade_maxima|     des_ped_idade|
+-----------------+------------+------------+------------------+
|29.69911764705882|        0.42|        80.0|14.526497332334035|
+-----------------+------------+------------+------------------+



In [14]:
#agregaçoes group by

spark.sql("""
  SELECT
    Sex,
    AVG(Age) as med_idade,
    MIN(Age) as idade_minima,
    MAX(Age) as idade_maxima,
    STDDEV(Age) as des_ped_idade
  FROM titanic
  GROUP BY Sex
""").show()

+------+------------------+------------+------------+------------------+
|   Sex|         med_idade|idade_minima|idade_maxima|     des_ped_idade|
+------+------------------+------------+------------+------------------+
|female|27.915708812260537|        0.75|        63.0|14.110146457544133|
|  male| 30.72664459161148|        0.42|        80.0|14.678200823816608|
+------+------------------+------------+------------+------------------+



In [16]:
spark.sql("""
  SELECT
    Sex, Pclass, Survived,
    AVG(Age) as med_idade,
    MIN(Age) as idade_minima,
    MAX(Age) as idade_maxima,
    STDDEV(Age) as des_ped_idade
  FROM titanic
  GROUP BY Sex,Pclass, Survived
""").show()

+------+------+--------+------------------+------------+------------+------------------+
|   Sex|Pclass|Survived|         med_idade|idade_minima|idade_maxima|     des_ped_idade|
+------+------+--------+------------------+------------+------------+------------------+
|female|     1|       0|25.666666666666668|         2.0|        50.0| 24.00694344004112|
|female|     2|       0|              36.0|        24.0|        57.0|12.915107432770352|
|  male|     2|       0| 33.36904761904762|        16.0|        70.0| 12.15812461947834|
|  male|     1|       0| 44.58196721311475|        18.0|        71.0|14.457748881905525|
|  male|     3|       0| 27.25581395348837|         1.0|        74.0|12.135706609773328|
|female|     3|       0|23.818181818181817|         2.0|        48.0|12.833464514612233|
|  male|     2|       1|            16.022|        0.67|        62.0|19.547121527222362|
|female|     2|       1|28.080882352941178|         2.0|        55.0| 12.76469311243358|
|  male|     1|      

In [17]:
spark.sql("""
  SELECT
    Sex, Pclass, Survived,
    AVG(Age) as med_idade,
    MIN(Age) as idade_minima,
    MAX(Age) as idade_maxima,
    STDDEV(Age) as des_ped_idade
  FROM titanic
  GROUP BY Sex,Pclass, Survived
  ORDER BY Sex,Pclass, Survived
""").show()

+------+------+--------+------------------+------------+------------+------------------+
|   Sex|Pclass|Survived|         med_idade|idade_minima|idade_maxima|     des_ped_idade|
+------+------+--------+------------------+------------+------------+------------------+
|female|     1|       0|25.666666666666668|         2.0|        50.0| 24.00694344004112|
|female|     1|       1|  34.9390243902439|        14.0|        63.0|13.223013572001038|
|female|     2|       0|              36.0|        24.0|        57.0|12.915107432770352|
|female|     2|       1|28.080882352941178|         2.0|        55.0| 12.76469311243358|
|female|     3|       0|23.818181818181817|         2.0|        48.0|12.833464514612233|
|female|     3|       1|19.329787234042552|        0.75|        63.0|12.303245681590393|
|  male|     1|       0| 44.58196721311475|        18.0|        71.0|14.457748881905525|
|  male|     1|       1|36.248000000000005|        0.92|        80.0|14.936743749765334|
|  male|     2|      